# Лекция 7. Динамическое программирование

> Динамическое программирование — это когда у нас есть задача, которую непонятно как решать, и мы разбиваем ее на меньшие задачи, которые тоже непонятно как решать.
<br><br>(с) А. Кумок.

Более формально - это построение алгоритма решения задачи через решение мелких подзадач с последующим конструированием решения исходной задачи.

Обычно вводится некое состояние системы $F(n_1, \ldots, n_k)$, которое является решением некой подзадачи. Данное состояние зависит некоторого набора параметров $\{n_k\}$. Если $k=1$, то говорят об одномерной задаче, если $k=2$ - двумерное и так далее. Фактически, поиск функциональной зависимости этого состояния от этих параметров и является основной задачей.

Чаще всего, оказывается, что это состояние является функцией других $M$ состояний

$$
F(n) = f(F(t_1), \ldots, F(t_M))
$$

Это и есть подзадача нашей задачи. Эту зависимость обычно выражают рекурсией.

## Шаг 0. Озарение.

Самый сложный шаг при решении задач методом динамического программирования - это понять, что задача может быть решена данным методом. На этом шаге очень помогает ознакомление с классическими задачами динамического программирования:
* Числа Фибоначи
* Задача о рюкзаке
* Задача об наибольшей подпоследовательности у двух последовательностей
* Задачи на шахматных досках
* и многие другие


Рассмотрим две задачи
1. **Числа фибоначи.** Дан ряд чисел, в этом ряде следующее число является суммой двух предыдущих. Например, $1, 1, 2, 3, 5, 8, 13, \ldots$
2. **Задача о пути.** Дано поле размером $N \times M$, в верхней левой части находится фигура. Она может перемещатся только направо и вниз. Нужно найти сколькими способами она может достигнуть нижней правой клетки.
<img src="https://miro.medium.com/max/800/0*zvQede7--TNKtpUO.png">

## Шаг 1. Рекурсия.

Именно на этом шаге мы должны найти наше состояние, как функцию некой величины. 

### Числа Фибоначи
Здесь все просто. Само число является функцией своего номера
$$
F(1) = 1
\\
F(2) = 1
\\
F(3) = 2
\\
F(4) = 3
\\
F(5) = 5
$$

При этом видно, что состояние с номером $n$ выражается через предыдущие два состояния

$$
F(n) = F(n-1) + F(n-2)
$$

Это выражение легко записать рекурсией. Обратите внимание, что здесь есть два первых состояния, которые не могут быть выражены через другие. Эти состояния задаются руками.

In [12]:
def F(n):
    if n <= 2:
        return 1
    return F(n-1) + F(n-2)

F(6)

8

### Задача о пути

А вот здесь уже посложнее. Здесь мы можем ввести состояние, которое указывает сколькими способами мы можем придти в клетку с координатами $(x, y)$. Для клеток ферхнего ряда всегда $F(x, 1) = 1$, для первого столбца $F(1, y) = 1$. 

Теперь нужно понять, как выразить общее состояние $F(x, y)$ через другие состояния. Если встать на клетку $(x, y)$, то можно заметить, что в нее можно придти исключительно только двумя путями: сверху и слева. Если мы знаем сколькими способами мы можем достигнуть левой и верхней клетки, то для текущей клетки - это их сумма.

$$
F(x, y) = F(x-1, y) + F(x, y-1)
$$

In [13]:
def F(x, y):
    if x == 1 or y == 1:
        return 1
    
    return F(x - 1, y) + F(x, y - 1)

F(8, 8)

3432

## Шаг 2. Запоминание.

В предыдущем шаге можно заметить, что не смотря на лаконичность решения - оно будет не эффективным. Мы много раза можем вычислять одни и теже состояния. Например, для чисел Фибоначи мы будем два раза вычислять состояния $F(3)$ для $F(5)$. Для того, чтобы такое не происходило - нужно запоминать состояния и использовать их повторно.

### Числа Фибоначи

Первый вариант самый простой, мы просто кешируем результаты вызова функции

In [15]:
from functools import lru_cache 

@lru_cache
def F(n):
    if n <= 2:
        return 1
    return F(n-1) + F(n-2)

F(6)

8

In [16]:
def Fib(N):
    dp = [None for _ in range(N)]

    def F(n):
        if dp[n-1] is not None: return dp[n-1]
    
        if n <= 2:
            dp[n-1] = 1            
        else:
            dp[n-1] = F(n-1) + F(n-2)        
            
        return dp[n-1]
    
    return F(N)

for n in range(1, 10):
    print(Fib(n))

1
1
2
3
5
8
13
21
34


### Задача о пути

Здесь все работает абсолютно также

In [18]:
from functools import lru_cache 

@lru_cache
def F(x, y):
    if x == 1 or y == 1:
        return 1
    
    return F(x - 1, y) + F(x, y - 1)

F(8, 8)

3432

In [17]:
def Count(X, Y):
    dp = [
        [None for y in range(Y)] for x in range(X)
    ]
    
    def F(x, y):
        if dp[x-1][y-1] is not None:
            return dp[x-1][y-1]
        
        if x == 1 or y == 1:
            dp[x-1][y-1] = 1
        else:
            dp[x-1][y-1] = F(x - 1, y) + F(x, y - 1)
            
        return dp[x-1][y-1]
            
    return F(X, Y)

Count(8, 8)

3432

### Шаг 3. Итерации.

У рекурсий есть один существенный недостаток - существует ограничение на предельную глубину рекурсии.

In [3]:
def F():
    return F()

F()

RecursionError: maximum recursion depth exceeded

In [2]:
import sys
print(sys.getrecursionlimit())

# Если очень хочется
sys.setrecursionlimit(1000)

1000


В этом случае приходится переходить к итерациям

### Числа Фибоначи

In [6]:
def Fib(N):
    if N <= 2: return 1
    dp = [None for n in range(N)]
    dp[0] = 1
    dp[1] = 1
    
    for n in range(2, N):
        dp[n] = dp[n-1] + dp[n-2]
        
    return dp[N-1]
    
    
Fib(6)

8

Можно обратить внимание, что нам не требуется хранить все возможные предыдущие состаяния, достаточно только последних двух

In [7]:
def Fib(N):
    if N <= 2: return 1

    dp = [1, 1]
        
    for n in range(2, N):
        dp[0], dp[1] = dp[1], dp[0] + dp[1]
        
    return dp[1]
    
    
Fib(6)

8

### Задача о пути

In [10]:
def Count(N, M):
    dp = [
        [None for y in range(M)]
        for x in range(N)
    ]
    
    for x in range(N):
        for y in range(M):
            if x == 0 or y == 0:
                dp[x][y] = 1
            else:
                dp[x][y] = dp[x-1][y] + dp[x][y-1]
            
    return dp[N-1][M-1]

Count(8, 8)

3432

Здесь также можно заметить, что нам по сути не нужно хранить все состояния, нам хватит одного списка (по сути изломанный вертикальный столбец)

In [4]:
def Count(N, M):
    dp = [1 for y in range(M)]
    
    for x in range(1, N):
        for y in range(1, M):
            dp[y] = dp[y] + dp[y-1]  
            
    return dp[-1]

Count(8, 8)

3432